In [2]:
import sys
sys.path.insert(0,'../../models/i3D/')

In [3]:
import numpy as np
import pandas as pd
import hashlib
import pickle
import glob
import cv2
import datetime
import os
from sklearn.model_selection import train_test_split
from soccer_functions import jupyter_animation
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.


In [4]:
file_name = './annotation_isportistics.xlsx'

In [5]:
#Loading excel sheet
df1 = pd.read_excel(file_name, sheet_name='fin_gol_assist_16-17')
df2 = pd.read_excel(file_name, sheet_name='fin_gol_assist_15-16')

#Removing unnecessary column to match dataframes
del df2['Tempo (jogo)']

#Match excel sheets in a single dataframe
df = pd.concat([df1, df2], ignore_index=True)

#Fixing typo error
df['Fundamento'] = df['Fundamento'].str.replace(' FF', 'FF', regex=True)

# Removing not finishings clips
df = df[(df['Fundamento'] != 'Assistência') &
   (df['Fundamento'] != 'Roubada de Bola')].reset_index(drop=True)

df['Periodo'] = df['Video'].map(lambda x: ''.join(i for i in x if i.isdigit()))

df['Tempo (segundos)'] = 0
#Remove row without time
for index, row in df.iterrows():
    value = row['Tempo (video)']
    if type(value) == str:
        try:
            value = datetime.datetime.strptime(value.strip(), '%H:%M:%S').time()
        except:
            pass
    if type(value) == datetime.time:

        df.loc[index, 'Tempo (segundos)'] = (value.hour * 60 + value.minute) * 60 + value.second
    else:
        df.drop(index)

In [6]:
file_name = 'finishings_dataset.pkl'
df.to_pickle(file_name)

In [7]:
df[df['Diretorio (jogo)'] == '2015-09-16 - 21-45 Gent 1 - 1 Lyon']

,JSON,Diretorio (jogo),Video,Fundamento,Tempo (video),Como,Ass.,Reb.,Pen.,Falta,Obs.:,Obs. Ass.,Desvio,Escanteio,Fin. Rast.,Espalmada,Periodo,Tempo (segundos)
502,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,DF,00:06:59,pe esquerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,NaN,1,419
503,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,TF,00:14:25,pe direito,NaN,NaN,NaN,NaN,Desvio no goleiro de acordo com o site da UEFA.,NaN,NaN,NaN,NaN,s,1,865
504,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,FF,00:18:42,pe esquerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1122
505,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,DF,00:22:00,pe esquerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,NaN,1,1320
506,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,DF,00:30:00,pe direito,NaN,NaN,NaN,NaN,dois tempos,NaN,NaN,NaN,NaN,s,1,1800
507,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,DF,00:31:44,pe direito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1904
508,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,DF,00:38:10,pe esquerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,s,1,2290
509,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,FF,00:38:35,cabeça,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2315
510,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,1.mkv,FF,00:43:41,pe direito,NaN,NaN,NaN,s,NaN,NaN,NaN,NaN,NaN,NaN,1,2621
511,europe_uefa-champions-league\2015-2016,2015-09-16 - 21-45 Gent 1 - 1 Lyon,2.mkv,DF,00:04:22,pe direito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,NaN,s,2,262


In [9]:
df[df['Diretorio (jogo)'] == '2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus']

,JSON,Diretorio (jogo),Video,Fundamento,Tempo (video),Como,Ass.,Reb.,Pen.,Falta,Obs.:,Obs. Ass.,Desvio,Escanteio,Fin. Rast.,Espalmada,Periodo,Tempo (segundos)
540,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,1.mkv,FF,00:15:49,pe direito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,949
541,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,1.mkv,DF,00:17:28,pe direito,NaN,NaN,NaN,NaN,dois tempos,NaN,NaN,NaN,s,s,1,1048
542,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,1.mkv,FF,00:28:19,pe esquerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,NaN,1,1699
543,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,1.mkv,Gol,00:38:12,pe direito,s,NaN,NaN,NaN,NaN,NaN,s,NaN,NaN,NaN,1,2292
544,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,1.mkv,Gol,00:40:42,pe direito,s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2442
545,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,1.mkv,Gol,00:45:38,mão,NaN,NaN,NaN,NaN,contra,NaN,NaN,NaN,NaN,NaN,1,2738
546,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,2.mkv,FF,00:18:19,pe direito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1099
547,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,2.mkv,DF,00:22:37,pe esquerdo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1357
548,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,2.mkv,DF,00:29:21,pe direito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,2,1761
549,europe_uefa-champions-league\2015-2016,2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus,2.mkv,DF,00:32:28,pe direito,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s,2,1948


In [ ]:
df[df['Diretorio (jogo)'] == '2015-09-29 - 21-45 Arsenal 2 - 3 Olympiakos Piraeus']

In [ ]:
def save_clips(df, videos_path, save_path, width=224, height=224, crop_lenght=4, fps = None):
    clip_filenames = df['Diretorio (jogo)'] + '_' + df['Tempo (segundos)'].astype(str) + '_' + df['Video'].str.split('.', expand=True)[0] 
    qty = len(df.index)
    past_video = 0
    for i, ID in enumerate(df.index):
#     for i, ID in enumerate([220, 255, 256]):
        filename = save_path + clip_filenames.loc[ID] + '.npy'
        print("%s/%s - ID: %i:"%(i,qty,ID),filename)
        if os.path.isfile(filename) == True:
            print('Done')
        else:
            try:
                video_path = glob.glob('./data/' + '/*/*/*' + df.loc[ID]['Diretorio (jogo)'] + '/' + df.loc[0]['Video'].split('.')[0] + '*')[0]
            except:
                print(video_path)
                print('Video not found!')
                continue
            if video_path != past_video:
                cap = cv2.VideoCapture(video_path)
                if type(fps) != int:
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    if fps > 30:
                        print(fps)
                        continue
            start_frame = int(df.loc[ID]['Tempo (segundos)']*fps)
            end_frame = int(start_frame + (crop_lenght*fps))
            selected_idx = np.linspace(start_frame, end_frame, 24).astype(int)
            selected_frames = np.empty(shape=(len(selected_idx), height, width, 3), dtype=np.uint8)
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            i = 0
            for frame_idx in range(start_frame, end_frame+1):
                frame = cap.read()[1]
                if frame_idx in selected_idx:
                    selected_frames[i] = cv2.resize(frame, (width,height))
                    i += 1
            directory = os.path.abspath(save_path + clip_filenames.loc[ID])
            np.save(directory, selected_frames)
            past_video = video_path
            print('Done')

In [ ]:
save_clips(df, videos_path='./data/', save_path='./clips/', width=224, height=224, crop_lenght=4)

In [ ]:
video_path = "./data/europe_uefa-champions-league/2016-2017/2017-03-08 - 22-45 Barcelona 6 - 1 Paris SG/2_HQ.mkv"
cap = cv2.VideoCapture(video_path)
num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = num_frames/ 5168.440000
print(fps)
print(num_frames)
cap.set(1, int(num_frames-1))
plt.figure(figsize=(15,15))
plt.imshow(cap.read()[1][...,::-1])

# Saving splits from full game

In [ ]:
video_length

In [ ]:
video_path = "./data/europe_uefa-champions-league/2016-2017/2016-09-13 - 21-45 Barcelona 7 - 0 Celtic/1_HQ.mkv"
main_directory = '/home/ubuntu/SoccerNet-code/data/splits/'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
clip_length = int(fps*4)
samples = 24
video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
height = 224
width = 224
video = np.zeros(shape=(video_length, height, width, 3), dtype=np.uint8)

print('Starting video resizing')
for i in range(video_length):
    video[i] = cv2.resize(cap.read()[1], (width,height))
    
splits_start = np.arange(0, video_length, 25)
print('Starting saving splits')
for i in splits_start:
    if i + clip_length < video_length:
        clip_name = str(i) + '_' + str(i + clip_length)
        directory = os.path.join(main_directory, clip_name)
        selected_idx = np.linspace(i, i + clip_length, samples).astype(int)
        selected_frames = video[selected_idx]
        np.save(directory, selected_frames)